In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import torch
from bs4 import BeautifulSoup
from pytorch_transformers import BertTokenizer, BertModel

import re

df = pd.read_csv('IMDB_Dataset.csv', encoding='utf-8')
# df = pd.read_csv('test.csv', encoding='utf-8')
# 시험삼아 해보세용..

In [ ]:
# Sentiment 1, 0으로 바꾸기
df['sentiment'] = (df['sentiment'] == 'positive').astype(int)

In [ ]:
# 문장 단위로 쪼개기
df_reviews = df.review.str.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', expand=True)
df_reviews = df_reviews.stack().reset_index(level=1, drop=True).to_frame('review_single')
df_pre = df[["sentiment"]].merge(df_reviews, left_index=True, right_index=True, how='left')
df_reviews = df_pre.values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# 전처리
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_review(raw_review: str) -> str:
    # 1. unicode to ASCII
    raw_review = unicodeToAscii(raw_review)
    
    # 2. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text()
    
    return review_text

def preprocess(review: str, total: int, show_progress: bool = True) -> list:
    if show_progress:
        global counter
        counter += 1
        if counter % 10000 == 0:
            print('Processing... %6i/%6i'% (counter, total), end='\r')
    # 1. Clean text
    review = clean_review(review)
    print(review)
        
    # 2. BERT
    token_ids = tokenizer.encode(review, add_special_tokens=True)
    print(tokenizer.decode(tokenizer.encode(review, add_special_tokens=True)))
    
    for token_id in token_ids:
        try:
            dictionary[token_id] += 1
        except:
            dictionary[token_id] = 1
    return

In [ ]:
counter = 0
count = 0

dictionary = {}

batch_size = 512
num = 1

df_length = len(df_reviews)
for df_review in df_reviews:
    preprocess(df_review[1], df_length)

In [ ]:
dict_df = pd.DataFrame(dictionary,index=[0]).T

In [ ]:
x = [(dictionary[i], i) for i in dictionary.keys()]

Top5000 = [x[i][1] for i in range(5000)]
Top5000_dist = {x[i][1] : x[i][0] for i in range(5000)}
Top5000_BtoA = {Top5000[i] : i for i in range(5000)}
Top5000_AtoB = {i : Top5000[i] for i in range(5000)}

In [ ]:
import pickle
import gzip

with gzip.open('Top5000_dist.pickle', 'wb') as f:
    pickle.dump(Top5000_dist, f)

with gzip.open('testPickleFile.pickle','rb') as f:
    data = pickle.load(f)